In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import sqlite3
import itertools

In [2]:
df = pd.read_csv('data/Crash_Reporting_-_Drivers_Data.csv', low_memory=False)

cols_to_drop = ['Report Number', 'Local Case Number','Latitude','Longitude',
                'Off-Road Description', 'Municipality','Related Non-Motorist',
                'Non-Motorist Substance Abuse', 'Circumstance']

df.drop(cols_to_drop, axis=1, inplace=True)

In [3]:
col_datatypes = {}

for i in df.columns:
    col_datatypes[i] = type(i)

In [4]:
d = {'Agency Name': str,
 'ACRS Report Type': str,
 'Crash Date/Time': 'datetime64[ns]',
 'Route Type': str,
 'Road Name': 'ignore',
 'Cross-Street Type': str,
 'Cross-Street Name': 'ignore',
 'Collision Type': str,
 'Weather': str,
 'Surface Condition': str,
 'Light': str,
 'Traffic Control': str,
 'Driver Substance Abuse': str,
 'Person ID': 'ignore',
 'Driver At Fault': str,
 'Injury Severity': str,
 'Driver Distracted By': str,
 'Drivers License State': str,
 'Vehicle ID': 'ignore',
 'Vehicle Damage Extent': str,
 'Vehicle First Impact Location': str,
 'Vehicle Second Impact Location': str,
 'Vehicle Body Type': str,
 'Vehicle Movement': str,
 'Vehicle Continuing Dir': str,
 'Vehicle Going Dir': str,
 'Speed Limit': float,
 'Driverless Vehicle': 'ignore',
 'Parked Vehicle': 'ignore',
 'Vehicle Year': float,
 'Vehicle Make': str,
 'Vehicle Model': str,
 'Equipment Problems': 'ignore',
 'Location': str}

In [5]:
for key, value in d.items():
    if value != 'ignore':
        df[key] = df[key].astype(value)
    else:
        df.drop(key, axis=1, inplace=True)

# df['speed_limit'] = df['Speed Limit']
df['Crash Date'] = df['Crash Date/Time'].dt.date
df['Crash Time'] = df['Crash Date/Time'].dt.time
df.drop('Crash Date/Time', axis=1, inplace=True)

In [6]:
df["Injury Severity"].value_counts()

NO APPARENT INJURY          140232
POSSIBLE INJURY              17367
SUSPECTED MINOR INJURY       11805
SUSPECTED SERIOUS INJURY      1411
FATAL INJURY                   153
Name: Injury Severity, dtype: int64

In [7]:
df["Driver At Fault"].replace({"Yes": 1, "No": 0, "Unknown": 0}, inplace=True)

In [8]:
df.columns = [col.replace(" ", "_").lower() for col in df.columns]

### SQL Queries

### 1. What is the sum, average, count of a given column

In [10]:
df.loc[df["vehicle_year"]==0.0, "vehicle_year"] = np.nan

In [11]:
df.loc[(df["vehicle_year"]<=2000) | (df["vehicle_year"] > pd.to_datetime("today").year), "vehicle_year"] = np.nan

df["vehicle_year"].value_counts(dropna=False)

NaN       15651
2015.0    11769
2014.0    11579
2016.0    11121
2013.0    10891
2012.0     9281
2017.0     9002
2011.0     8335
2007.0     7888
2008.0     7866
2010.0     7390
2006.0     7390
2009.0     6757
2018.0     6673
2005.0     6477
2004.0     6128
2019.0     5405
2003.0     5341
2002.0     4263
2001.0     3472
2020.0     3424
2021.0     2529
2022.0     1608
2023.0      679
2024.0       49
Name: vehicle_year, dtype: int64

In [12]:
# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
# Insert the Pandas DataFrame into the SQLite database
df.to_sql('crash_data', conn, index=False)
num_functions = ['avg', 'min', 'max', 'sum']

for func in num_functions:
    for col in df.select_dtypes(include=['float','int']).columns:
        query = f"SELECT {func}({col}) as {col} from crash_data"
        print(f"what is the {func} {col}: {pd.read_sql_query(query, conn).iat[0, 0]}")

what is the avg driver_at_fault: 0.5371414533713912
what is the avg speed_limit: 32.55188105376445
what is the avg vehicle_year: 2011.5437009470952
what is the min driver_at_fault: 0
what is the min speed_limit: 0.0
what is the min vehicle_year: 2001.0
what is the max driver_at_fault: 1
what is the max speed_limit: 75.0
what is the max vehicle_year: 2024.0
what is the sum driver_at_fault: 91834
what is the sum speed_limit: 5565330.0
what is the sum vehicle_year: 312426933.0


In [15]:
cat_cols = ["vehicle_make", "injury_severity", "vehicle_continuing_dir"]
num_cols = ["speed_limit", "vehicle_year"]
date_cols = ["crash_date"]
bin_cols = ["driver_at_fault"]

"""
categorical: by category level
numerical: by average, max, min and total
date: by year, by month
binary: when present or not (x 2) for every binary

#2 dim
num with num        : what is avg vehicle year and avg speed limit
num with binary     : what is avg speed limit when driver was (at fault / not at fault)
cat with binary     : what is injury severity when driver was at fault
cat with cat        : what is injury severity by vehicle make
num with cat        : what is avg speed limit by injury severity
num with date       : what is avg speed limit by month/year

cat with date       : what is vehicle make by month
binary with date    : how many times driver was at fault by year

#3 give some thought about 3 dimensions (limit)
#4 check with some other clean datasets

"""

# for f in num_functions:
#     for num in num_cols:
#         print(f"what is {f} ")

'\ncategorical: by category level\nnumerical: by average, max, min and total\ndate: by year, by month\nbinary: when present or not (x 2) for every binary\n\n#2 dim\nnum with num        : what is avg vehicle year and avg speed limit\n\nKaran (make 3 cells with dynamic for loop covering all the questions):\nnum with binary     : what is avg speed limit when driver was (at fault / not at fault)\ncat with binary     : what is injury severity when driver was at fault\nnum with cat        : what is avg speed limit by injury severity\n\nShantanu (make 4 cells with dynamic for loop covering all the questions)::\nnum with date       : what is avg speed limit by month/year\ncat with cat        : what is injury severity by vehicle make\ncat with date       : what is vehicle make by month\nbinary with date    : how many times driver was at fault by year\n\n#3 give some thought about 3 dimensions (limit)\n#4 check with some other clean datasets\n\n'

In [16]:
list(itertools.product(num_cols, bin_cols))

[('speed_limit', 'driver_at_fault'), ('vehicle_year', 'driver_at_fault')]

In [17]:
# what is the average speed limit
col = "speed_limit"
query = f"SELECT avg({col}) as average_speed from crash_data"
pd.read_sql_query(query, conn).iat[0, 0]

32.55188105376445

In [18]:
combinations = list(itertools.combinations(num_cols, 2))
num_functions = ['min', 'max', 'avg', 'total']
all_perm = list(itertools.permutations(num_functions, 2))
all_perm.extend([(f, f) for f in num_functions])

In [26]:
from copy import copy
combinations = list(itertools.combinations(num_cols, 2))

num_functions = ['min', 'max', 'avg', 'total']
all_perm = list(itertools.permutations(num_functions, 2))
full_perm = copy(all_perm)
full_perm.extend([(f, f) for f in num_functions])

# for just one numerical feature
if len(num_cols) == 1:
    for f1, f2 in all_perm:
        for c in num_cols:
            query = f"SELECT {f1}({c}) as {f1}_{c}, {f2}({c}) as {f2}_{c} from crash_data"
            print(f"what is {f1} {c} and {f2} {c}: {pd.read_sql_query(query, conn)}")

else:
    # numerical with numerical
    for f1, f2 in all_perm:
        for c1, c2 in combinations:
            query1 = f"SELECT {f1}({c1}) as {f1}_{c1}, {f2}({c1}) as {f2}_{c1} from crash_data"
            print(f"what is {f1} {c1} and {f2} {c1}: {pd.read_sql_query(query1, conn)}")
            query2 = f"SELECT {f1}({c2}) as {f1}_{c2}, {f2}({c2}) as {f2}_{c2} from crash_data"
            print(f"what is {f1} {c2} and {f2} {c2}: {pd.read_sql_query(query2, conn)}")
            query3 = f"SELECT {f1}({c1}) as {f1}_{c1}, {f2}({c2}) as {f2}_{c2} from crash_data"
            print(f"what is {f1} {c1} and {f2} {c2}: {pd.read_sql_query(query3, conn)}")
            query4 = f"SELECT {f1}({c2}) as {f1}_{c2}, {f2}({c1}) as {f2}_{c1} from crash_data"
            print(f"what is {f1} {c2} and {f2} {c1}: {pd.read_sql_query(query4, conn)}")

# numerical with binary
for func in num_functions:
    for col in num_cols:
        for cond in bin_cols:
            print(f"what is {func} {col} when {cond}")
            print(f"what is {func} {col} when not {cond}")

# categorical with binary
for col in cat_cols:
    for cond in bin_cols:
        print(f"what {col} has most counts when {cond}")
        print(f"what {col} has least counts when {cond}")
        for n in range(1, 3):
            print(f"what are the top {n+1} {col} when {cond}")

# categorical with categorical
for i in cat_cols:
    for j in cat_cols:
        if j!= i:
            print(f"what is the most {j} by {i}")
            print(f"what is the least {j} by {i}")

# numerical with categorical
for func in num_functions:
    for num_col in num_cols:
        for cat_col in cat_cols:
            print(f"what is {func} {num_col} by {cat_col}")

# numerical with date
for func in num_functions:
    for num_col in num_cols:
        for date_col in date_cols:
            print(f"what is {func} {num_col} by {date_col} month")
            print(f"what is {func} {num_col} by {date_col} year")

# categorical with date
for cat_col in cat_cols:
    for date_col in date_cols:
        print(f"what is most {cat_col} by {date_col} month")
        print(f"what is least {cat_col} by {date_col} year")
        print(f"what is most {cat_col} by {date_col} month")
        print(f"what is least {cat_col} by {date_col} year")
        for n in range(1, 3):
            print(f"what are the top {n+1} {cat_col} by month")
            print(f"what are the top {n+1} {cat_col} by year")

# categorical with binary


what is min speed_limit and max speed_limit:    min_speed_limit  max_speed_limit
0              0.0             75.0
what is min vehicle_year and max vehicle_year:    min_vehicle_year  max_vehicle_year
0            2001.0            2024.0
what is min speed_limit and max vehicle_year:    min_speed_limit  max_vehicle_year
0              0.0            2024.0
what is min vehicle_year and max speed_limit:    min_vehicle_year  max_speed_limit
0            2001.0             75.0
what is min speed_limit and avg speed_limit:    min_speed_limit  avg_speed_limit
0              0.0        32.551881
what is min vehicle_year and avg vehicle_year:    min_vehicle_year  avg_vehicle_year
0            2001.0       2011.543701
what is min speed_limit and avg vehicle_year:    min_speed_limit  avg_vehicle_year
0              0.0       2011.543701
what is min vehicle_year and avg speed_limit:    min_vehicle_year  avg_speed_limit
0            2001.0        32.551881
what is min speed_limit and total speed_

In [41]:
pd.read_sql_query(query, conn)

,total_speed_limit,avg_speed_limit
0,5565330.0,32.551881


In [38]:
df['month_year'] = df[""]

0         2019-05-31
1         2023-07-21
2         2023-07-20
3         2023-07-23
4         2023-07-24
             ...    
170963    2016-03-01
170964    2017-07-19
170965    2020-11-23
170966    2019-11-23
170967    2015-01-21
Name: crash_date, Length: 170968, dtype: object

In [47]:
num_functions

['min', 'max', 'avg', 'total']

what is most vehicle_make by crash_date month
what is least vehicle_make by crash_date year
what is most vehicle_make by crash_date month
what is least vehicle_make by crash_date year
what are the top 2 vehicle_make by month
what are the top 2 vehicle_make by year
what are the top 3 vehicle_make by month
what are the top 3 vehicle_make by year
what is most injury_severity by crash_date month
what is least injury_severity by crash_date year
what is most injury_severity by crash_date month
what is least injury_severity by crash_date year
what are the top 2 injury_severity by month
what are the top 2 injury_severity by year
what are the top 3 injury_severity by month
what are the top 3 injury_severity by year
what is most vehicle_continuing_dir by crash_date month
what is least vehicle_continuing_dir by crash_date year
what is most vehicle_continuing_dir by crash_date month
what is least vehicle_continuing_dir by crash_date year
what are the top 2 vehicle_continuing_dir by month
what are 

In [ ]:
# Karan (make 3 cells with dynamic for loop covering all the questions):
# cat with binary     : what is injury severity when driver was at fault
# num with cat        : what is avg speed limit by injury severity
#
# Shantanu (make 4 cells with dynamic for loop covering all the questions)::
# num with date       : what is avg speed limit by month/year
# cat with cat        : what is injury severity by vehicle make
# cat with date       : what is vehicle make by month
# binary with date    : how many times driver was at fault by year

In [29]:
cat_cols, bin_cols

(['vehicle_make', 'injury_severity', 'vehicle_continuing_dir'],
 ['driver_at_fault'])

In [32]:
df["vehicle_continuing_dir"].value_counts()

North      46536
South      45634
East       35856
West       34655
Unknown     5621
nan         2666
Name: vehicle_continuing_dir, dtype: int64

In [ ]:
# What vehicle_make has most counts when driver at fault
# What was the most injury_severity when driver at fault
# What was the vehicle_continuing_dir when driver at fault

# What vehicle_make has most counts when driver at no fault
# What was the most injury_severity when driver at no fault
# What was the vehicle_continuing_dir when driver at no fault

# What are the top two vehicle_make when driver at fault
# What was the top two injury_severity when driver at fault
# What are the top two vehicle_make when driver at no fault
# What are the top two injury_severity when driver at no fault

what vehicle_make has most counts when driver_at_fault
what vehicle_make has least counts when driver_at_fault
what are the top 2 vehicle_make when driver_at_fault
what are the top 3 vehicle_make when driver_at_fault
what injury_severity has most counts when driver_at_fault
what injury_severity has least counts when driver_at_fault
what are the top 2 injury_severity when driver_at_fault
what are the top 3 injury_severity when driver_at_fault
what vehicle_continuing_dir has most counts when driver_at_fault
what vehicle_continuing_dir has least counts when driver_at_fault
what are the top 2 vehicle_continuing_dir when driver_at_fault
what are the top 3 vehicle_continuing_dir when driver_at_fault


what is min speed_limit when driver_at_fault
what is min speed_limit when not driver_at_fault
what is min vehicle_year when driver_at_fault
what is min vehicle_year when not driver_at_fault
what is max speed_limit when driver_at_fault
what is max speed_limit when not driver_at_fault
what is max vehicle_year when driver_at_fault
what is max vehicle_year when not driver_at_fault
what is avg speed_limit when driver_at_fault
what is avg speed_limit when not driver_at_fault
what is avg vehicle_year when driver_at_fault
what is avg vehicle_year when not driver_at_fault
what is total speed_limit when driver_at_fault
what is total speed_limit when not driver_at_fault
what is total vehicle_year when driver_at_fault
what is total vehicle_year when not driver_at_fault


In [28]:
num_functions

['min', 'max', 'avg', 'total']

In [ ]:
what is avg and min speed
what is avg and min year
what is avg year and min speed
what is avg speed and min year
what is avg and min speed
what is avg and min cost
what is avg cost and min speed
what is avg speed and min cost
what is avg and min year
what is avg and min cost
what is avg cost and min year
what is avg year and min cost
what is avg and max speed
what is avg and max year
what is avg year and max speed
what is avg speed and max year
what is avg and max speed
what is avg and max cost
what is avg cost and max speed
what is avg speed and max cost
what is avg and max year
what is avg and max cost
what is avg cost and max year
what is avg year and max cost
what is avg and sum speed
what is avg and sum year
what is avg year and sum speed
what is avg speed and sum year
what is avg and sum speed
what is avg and sum cost
what is avg cost and sum speed
what is avg speed and sum cost
what is avg and sum year
what is avg and sum cost
what is avg cost and sum year
what is avg year and sum cost
what is min and avg speed
what is min and avg year
what is min year and avg speed
what is min speed and avg year
what is min and avg speed
what is min and avg cost
what is min cost and avg speed
what is min speed and avg cost
what is min and avg year
what is min and avg cost
what is min cost and avg year
what is min year and avg cost
what is min and max speed
what is min and max year
what is min year and max speed
what is min speed and max year
what is min and max speed
what is min and max cost
what is min cost and max speed
what is min speed and max cost
what is min and max year
what is min and max cost
what is min cost and max year
what is min year and max cost
what is min and sum speed
what is min and sum year
what is min year and sum speed
what is min speed and sum year
what is min and sum speed
what is min and sum cost
what is min cost and sum speed
what is min speed and sum cost
what is min and sum year
what is min and sum cost
what is min cost and sum year
what is min year and sum cost
what is max and avg speed
what is max and avg year
what is max year and avg speed
what is max speed and avg year
what is max and avg speed
what is max and avg cost
what is max cost and avg speed
what is max speed and avg cost
what is max and avg year
what is max and avg cost
what is max cost and avg year
what is max year and avg cost
what is max and min speed
what is max and min year
what is max year and min speed
what is max speed and min year
what is max and min speed
what is max and min cost
what is max cost and min speed
what is max speed and min cost
what is max and min year
what is max and min cost
what is max cost and min year
what is max year and min cost
what is max and sum speed
what is max and sum year
what is max year and sum speed
what is max speed and sum year
what is max and sum speed
what is max and sum cost
what is max cost and sum speed
what is max speed and sum cost
what is max and sum year
what is max and sum cost
what is max cost and sum year
what is max year and sum cost
what is sum and avg speed
what is sum and avg year
what is sum year and avg speed
what is sum speed and avg year
what is sum and avg speed
what is sum and avg cost
what is sum cost and avg speed
what is sum speed and avg cost
what is sum and avg year
what is sum and avg cost
what is sum cost and avg year
what is sum year and avg cost
what is sum and min speed
what is sum and min year
what is sum year and min speed
what is sum speed and min year
what is sum and min speed
what is sum and min cost
what is sum cost and min speed
what is sum speed and min cost
what is sum and min year
what is sum and min cost
what is sum cost and min year
what is sum year and min cost
what is sum and max speed
what is sum and max year
what is sum year and max speed
what is sum speed and max year
what is sum and max speed
what is sum and max cost
what is sum cost and max speed
what is sum speed and max cost
what is sum and max year
what is sum and max cost
what is sum cost and max year
what is sum year and max cost
what is avg and avg speed
what is avg and avg year
what is avg year and avg speed
what is avg speed and avg year
what is avg and avg speed
what is avg and avg cost
what is avg cost and avg speed
what is avg speed and avg cost
what is avg and avg year
what is avg and avg cost
what is avg cost and avg year
what is avg year and avg cost
what is min and min speed
what is min and min year
what is min year and min speed
what is min speed and min year
what is min and min speed
what is min and min cost
what is min cost and min speed
what is min speed and min cost
what is min and min year
what is min and min cost
what is min cost and min year
what is min year and min cost
what is max and max speed
what is max and max year
what is max year and max speed
what is max speed and max year
what is max and max speed
what is max and max cost
what is max cost and max speed
what is max speed and max cost
what is max and max year
what is max and max cost
what is max cost and max year
what is max year and max cost
what is sum and sum speed
what is sum and sum year
what is sum year and sum speed
what is sum speed and sum year
what is sum and sum speed
what is sum and sum cost
what is sum cost and sum speed
what is sum speed and sum cost
what is sum and sum year
what is sum and sum cost
what is sum cost and sum year
what is sum year and sum cost
​